In [23]:
from pathlib import Path
from src.utils.data_version_manager import DataVersionManager

# 1) สร้างอ็อบเจ็กต์ — ระบุโฟลเดอร์เก็บข้อมูลดิบและโฟลเดอร์เก็บ FAISS index
dvm = DataVersionManager(
    base_data_dir="data",          # จะมี data/raw/ เกิดขึ้น
    base_index_dir="artifacts",    # จะมี artifacts/faiss_index_vX.Y/
    data_version="latest",         # ค่าเริ่มต้น = "latest"
    gcs_path=None                  # ใส่ 'gs://bucket/data/raw' ถ้าจะใช้ GCS
)

# 2) เพิ่มเวอร์ชันข้อมูลใหม่
raw_sources = ["docs/txts/a.txt", "docs/txts/b.txt"]
new_version: str = dvm.create_new_version(raw_sources, increment_type="minor")
# → สร้าง data/raw/v1.1/, คัดลอกไฟล์ และอัปเดต symlink data/raw/latest →

Source file docs/txts/a.txt does not exist
Source file docs/txts/b.txt does not exist


In [24]:
dvm.list_available_versions()

['v1.0', 'v1.1', 'v1.2', 'v1.3']

In [25]:
# 3) ดึง path ของไฟล์ดิบตามเวอร์ชันที่ต้องการ
data_dir: Path = dvm.get_data_version_path("latest")  # หรือ "v1.0" ฯลฯ
data_dir

PosixPath('data/raw/latest')

In [26]:
 
# 4) (ขั้นตอนของคุณ) โหลดไฟล์ดิบ, สร้าง embedding แล้ว build FAISS index
index_dir: Path = dvm.get_index_path_for_version(new_version)
index_path = index_dir / "index.faiss"
# ... สร้างและบันทึก index.faiss ลงไปที่ index_path ...

# 5) สร้าง lineage record ไว้สืบย้อนกลับ
lineage = dvm.create_lineage_record(
    index_path=str(index_path),
    data_version=new_version,
    files_used=raw_sources,
    parameters={
        "embedding_model": "all-MiniLM-L6-v2",
        "chunk_size": 512,
        "faiss_dim": 384
    }
)
# artifacts/faiss_index_v1.1/lineage.json จะถูกเขียนอัตโนมัติ

# 6) ภายหลังสามารถอ่าน lineage ย้อนดูได้
info = dvm.get_lineage_for_index(str(index_path))
print(info["files"][0]["md5_hash"])

IndexError: list index out of range